In [104]:
import os

In [105]:

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import json
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from PyPDF2 import PdfReader


In [106]:
load_dotenv()


True

In [107]:
key=os.getenv("OPENAI_API_KEY")

In [108]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B48E403DD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B48E5EDDF0>, root_client=<openai.OpenAI object at 0x000001B48E16F740>, root_async_client=<openai.AsyncOpenAI object at 0x000001B48E0D9160>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [109]:
with open("C:/projects/python/GenAi/MCQ-Generator/GenAi/response.json", "r") as f:
   RESPONSE_JSON = json.load(f)
print(RESPONSE_JSON)   


{'1': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [110]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [111]:
quiz_genration=PromptTemplate(
  input_variables=["text","number","subject","tone","number","RESPONSE_JSON"],
  template=TEMPLATE
)

In [112]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_genration,output_key="quiz",verbose=True)

In [113]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [114]:
#quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)
quiz_evaluation_prompt= PromptTemplate(
  input_variables=["subject", "quiz"],
  template=TEMPLATE2
) 


In [115]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [116]:
genrate_evaluation_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","number","RESPONSE_JSON"],output_variables=["quiz","review"],verbose=True)

In [117]:
PATH = "C:/projects/python/GenAi/MCQ-Generator/GenAi/data.txt"
print(PATH)


C:/projects/python/GenAi/MCQ-Generator/GenAi/data.txt


In [118]:
with open(PATH,"r")as file:
  TEXT=file.read()
print(TEXT)  

Generative AI relies on sophisticated machine learning models called deep learning modelsâ€”algorithms that simulate the learning and decision-making processes of the human brain. These models work by identifying and encoding the patterns and relationships in huge amounts of data, and then using that information to understand users' natural language requests or questions and respond with relevant new content.

AI has been a hot technology topic for the past decade, but generative AI, and specifically the arrival of ChatGPT in 2022, has thrust AI into worldwide headlines and launched an unprecedented surge of AI innovation and adoption. Generative AI offers enormous productivity benefits for individuals and organizations, and while it also presents very real challenges and risks, businesses are forging ahead, exploring how the technology can improve their internal workflows and enrich their products and services. According to research by the management consulting firm McKinsey, one thir

In [119]:
genrate_evaluation_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{RESPONSE_JSON}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B48E403DD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B48E5EDDF0>, root_client=<openai.OpenAI object at 0x000001B48E16F740>, root_async_client=<openai.AsyncOpenAI object at 0x000001B48E0D9160>, temperature

In [120]:
TEXT,
NUMBER=3,
SUBJECT="AI",
TONE="Simple",
RESPONSE_JSON=RESPONSE_JSON

In [121]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [122]:
with get_openai_callback() as cb:
  res=genrate_evaluation_chain(
    {
      "text":TEXT,
    "number":NUMBER,
    "subject":SUBJECT,
    "tone":TONE,
    "RESPONSE_JSON":json.dumps(RESPONSE_JSON)
    }
  )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Generative AI relies on sophisticated machine learning models called deep learning modelsâ€”algorithms that simulate the learning and decision-making processes of the human brain. These models work by identifying and encoding the patterns and relationships in huge amounts of data, and then using that information to understand users' natural language requests or questions and respond with relevant new content.

AI has been a hot technology topic for the past decade, but generative AI, and specifically the arrival of ChatGPT in 2022, has thrust AI into worldwide headlines and launched an unprecedented surge of AI innovation and adoption. Generative AI offers enormous productivity benefits for individuals and organizations, and while it also presents very real challenges and risks, businesses are forging ahead, exploring how the technology can improve their internal workflows and en

In [123]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 1247
Prompt Tokens: 934
Completion Tokens: 313
Total Cost (USD): $0.0009364999999999999


In [124]:
print(res)

{'text': "Generative AI relies on sophisticated machine learning models called deep learning modelsâ€”algorithms that simulate the learning and decision-making processes of the human brain. These models work by identifying and encoding the patterns and relationships in huge amounts of data, and then using that information to understand users' natural language requests or questions and respond with relevant new content.\n\nAI has been a hot technology topic for the past decade, but generative AI, and specifically the arrival of ChatGPT in 2022, has thrust AI into worldwide headlines and launched an unprecedented surge of AI innovation and adoption. Generative AI offers enormous productivity benefits for individuals and organizations, and while it also presents very real challenges and risks, businesses are forging ahead, exploring how the technology can improve their internal workflows and enrich their products and services. According to research by the management consulting firm McKins

In [125]:
quiz=res.get("quiz")

In [126]:
quiz=json.loads(quiz)

In [127]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [128]:
quiz_table_data

[{'MCQ': 'What type of machine learning models does generative AI rely on?',
  'Choices': 'a: Supervised learning models | b: Deep learning models | c: Reinforcement learning models | d: Unsupervised learning models',
  'Correct': 'b'},
 {'MCQ': 'According to McKinsey research, what percentage of organizations are already using generative AI regularly in at least one business function?',
  'Choices': 'a: 10% | b: 50% | c: 33% | d: 75%',
  'Correct': 'c'},
 {'MCQ': 'What are the most common foundation models used in generative AI applications today?',
  'Choices': 'a: Image models | b: Large language models (LLMs) | c: Reinforcement learning models | d: Supervised learning models',
  'Correct': 'b'}]

In [131]:
df=pd.DataFrame(quiz_table_data)

In [133]:
df.to_csv("genai.csv",index=False)